<a href="https://colab.research.google.com/github/RyuichiSaito1/inflation-reddit-usa/blob/main/src/roberta_and_deberta_performances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install transformers
!pip install torch
!pip install accelerate
!pip install --upgrade accelerate
# After installing, restart the runtime.

In [ ]:

import torch

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Training + validation: 1040

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

# # Function to read data from TSV file using pandas
# def read_tsv(file_path):
#     data = pd.read_table(file_path, names=['body', 'inflation'], header=0, dtype='object', engine='python')
#     return data

def read_csv_file(file_path):
    try:
        data = pd.read_csv(file_path, names=['body', 'inflation'], header=0, dtype='object')
        return data
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Test data file path (Replace with your Google Drive directory and file)
file_path = '/content/drive/MyDrive/world-inflation/data/reddit/production/test-data-200.csv'

# Read data from TSV file using pandas
test_data = read_csv_file(file_path)

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

# Encode the test data
test_encodings = tokenizer(test_data['body'].tolist(), truncation=True, padding=True)

# Convert the string labels to integers
test_labels = [int(label) for label in test_data['inflation']]

# Create the test dataset
test_dataset = TestDataset(test_encodings, test_labels)

# Initialize the model
model = RobertaForSequenceClassification.from_pretrained('/content/drive/MyDrive/world-inflation/data/model/roberta-large-fine-tuning/checkpoint-392/')

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=16)

# Lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Use the model to predict the labels of the test data
for batch in test_loader:
    inputs = {key: val.to(model.device) for key, val in batch.items() if key != 'labels'}
    labels = batch['labels'].to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    true_labels.extend(labels.tolist())
    predicted_labels.extend(predictions.tolist())

# Calculate and display accuracy, recall, precision, and F1 score
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average=None)
precision = precision_score(true_labels, predicted_labels, average=None)
f1 = f1_score(true_labels, predicted_labels, average=None)

# Display classification report and confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

# Display metrics for each class and macro/micro averages
macro_avg = precision.mean(), recall.mean(), f1.mean()
micro_avg = precision.sum() / 3, recall.sum() / 3, f1.sum() / 3

# Display metrics for each class and macro/micro averages
print("+--------------+-----------+----------+----------+----------+")
print("|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |")
print("+--------------+-----------+----------+----------+----------+")
for i in range(3):
    print(f"| Class {i}      |    {accuracy:.2f}   |   {recall[i]:.2f}   |   {precision[i]:.2f}   |   {f1[i]:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Macro Average|    {accuracy:.2f}   |   {recall.mean():.2f}   |   {precision.mean():.2f}   |   {f1.mean():.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Micro Average|    {accuracy:.2f}   |   {recall.sum()/3:.2f}   |   {precision.sum()/3:.2f}   |   {f1.sum()/3:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")


Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.92      0.86        62
           1       0.89      0.71      0.79        77
           2       0.79      0.87      0.83        61

    accuracy                           0.82       200
   macro avg       0.83      0.83      0.83       200
weighted avg       0.83      0.82      0.82       200


Confusion Matrix:
[[57  4  1]
 [ 9 55 13]
 [ 5  3 53]]
+--------------+-----------+----------+----------+----------+
|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |
+--------------+-----------+----------+----------+----------+
| Class 0      |    0.82   |   0.92   |   0.80   |   0.86   |
| Class 1      |    0.82   |   0.71   |   0.89   |   0.79   |
| Class 2      |    0.82   |   0.87   |   0.79   |   0.83   |
+--------------+-----------+----------+----------+----------+
| Macro Average|    0.82   |   0.83   |   0.83   |   0.83   |
+--------------+-----------+----------+----

# Training + validation: 520

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

# # Function to read data from TSV file using pandas
# def read_tsv(file_path):
#     data = pd.read_table(file_path, names=['body', 'inflation'], header=0, dtype='object', engine='python')
#     return data

def read_csv_file(file_path):
    try:
        data = pd.read_csv(file_path, names=['body', 'inflation'], header=0, dtype='object')
        return data
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Test data file path (Replace with your Google Drive directory and file)
file_path = '/content/drive/MyDrive/world-inflation/data/reddit/production/test-data-200.csv'

# Read data from TSV file using pandas
test_data = read_csv_file(file_path)

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

# Encode the test data
test_encodings = tokenizer(test_data['body'].tolist(), truncation=True, padding=True)

# Convert the string labels to integers
test_labels = [int(label) for label in test_data['inflation']]

# Create the test dataset
test_dataset = TestDataset(test_encodings, test_labels)

# Initialize the model
model = RobertaForSequenceClassification.from_pretrained('/content/drive/MyDrive/world-inflation/data/model/roberta-large-fine-tuning-520/checkpoint-392/')

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=16)

# Lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Use the model to predict the labels of the test data
for batch in test_loader:
    inputs = {key: val.to(model.device) for key, val in batch.items() if key != 'labels'}
    labels = batch['labels'].to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    true_labels.extend(labels.tolist())
    predicted_labels.extend(predictions.tolist())

# Calculate and display accuracy, recall, precision, and F1 score
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average=None)
precision = precision_score(true_labels, predicted_labels, average=None)
f1 = f1_score(true_labels, predicted_labels, average=None)

# Display classification report and confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

# Display metrics for each class and macro/micro averages
macro_avg = precision.mean(), recall.mean(), f1.mean()
micro_avg = precision.sum() / 3, recall.sum() / 3, f1.sum() / 3

# Display metrics for each class and macro/micro averages
print("+--------------+-----------+----------+----------+----------+")
print("|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |")
print("+--------------+-----------+----------+----------+----------+")
for i in range(3):
    print(f"| Class {i}      |    {accuracy:.2f}   |   {recall[i]:.2f}   |   {precision[i]:.2f}   |   {f1[i]:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Macro Average|    {accuracy:.2f}   |   {recall.mean():.2f}   |   {precision.mean():.2f}   |   {f1.mean():.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Micro Average|    {accuracy:.2f}   |   {recall.sum()/3:.2f}   |   {precision.sum()/3:.2f}   |   {f1.sum()/3:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")


Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.82      0.76        62
           1       0.75      0.57      0.65        77
           2       0.71      0.79      0.74        61

    accuracy                           0.71       200
   macro avg       0.72      0.73      0.72       200
weighted avg       0.72      0.71      0.71       200


Confusion Matrix:
[[51  9  2]
 [15 44 18]
 [ 7  6 48]]
+--------------+-----------+----------+----------+----------+
|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |
+--------------+-----------+----------+----------+----------+
| Class 0      |    0.71   |   0.82   |   0.70   |   0.76   |
| Class 1      |    0.71   |   0.57   |   0.75   |   0.65   |
| Class 2      |    0.71   |   0.79   |   0.71   |   0.74   |
+--------------+-----------+----------+----------+----------+
| Macro Average|    0.71   |   0.73   |   0.72   |   0.72   |
+--------------+-----------+----------+----

# Training + validation: 260

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

# # Function to read data from TSV file using pandas
# def read_tsv(file_path):
#     data = pd.read_table(file_path, names=['body', 'inflation'], header=0, dtype='object', engine='python')
#     return data

def read_csv_file(file_path):
    try:
        data = pd.read_csv(file_path, names=['body', 'inflation'], header=0, dtype='object')
        return data
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Test data file path (Replace with your Google Drive directory and file)
file_path = '/content/drive/MyDrive/world-inflation/data/reddit/production/test-data-200.csv'

# Read data from TSV file using pandas
test_data = read_csv_file(file_path)

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

# Encode the test data
test_encodings = tokenizer(test_data['body'].tolist(), truncation=True, padding=True)

# Convert the string labels to integers
test_labels = [int(label) for label in test_data['inflation']]

# Create the test dataset
test_dataset = TestDataset(test_encodings, test_labels)

# Initialize the model
model = RobertaForSequenceClassification.from_pretrained('/content/drive/MyDrive/world-inflation/data/model/roberta-large-fine-tuning-260/checkpoint-147/')

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=16)

# Lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Use the model to predict the labels of the test data
for batch in test_loader:
    inputs = {key: val.to(model.device) for key, val in batch.items() if key != 'labels'}
    labels = batch['labels'].to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    true_labels.extend(labels.tolist())
    predicted_labels.extend(predictions.tolist())

# Calculate and display accuracy, recall, precision, and F1 score
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average=None)
precision = precision_score(true_labels, predicted_labels, average=None)
f1 = f1_score(true_labels, predicted_labels, average=None)

# Display classification report and confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

# Display metrics for each class and macro/micro averages
macro_avg = precision.mean(), recall.mean(), f1.mean()
micro_avg = precision.sum() / 3, recall.sum() / 3, f1.sum() / 3

# Display metrics for each class and macro/micro averages
print("+--------------+-----------+----------+----------+----------+")
print("|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |")
print("+--------------+-----------+----------+----------+----------+")
for i in range(3):
    print(f"| Class {i}      |    {accuracy:.2f}   |   {recall[i]:.2f}   |   {precision[i]:.2f}   |   {f1[i]:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Macro Average|    {accuracy:.2f}   |   {recall.mean():.2f}   |   {precision.mean():.2f}   |   {f1.mean():.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Micro Average|    {accuracy:.2f}   |   {recall.sum()/3:.2f}   |   {precision.sum()/3:.2f}   |   {f1.sum()/3:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")


Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.03      0.06        62
           1       0.40      1.00      0.57        77
           2       0.83      0.08      0.15        61

    accuracy                           0.42       200
   macro avg       0.74      0.37      0.26       200
weighted avg       0.72      0.42      0.29       200


Confusion Matrix:
[[ 2 59  1]
 [ 0 77  0]
 [ 0 56  5]]
+--------------+-----------+----------+----------+----------+
|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |
+--------------+-----------+----------+----------+----------+
| Class 0      |    0.42   |   0.03   |   1.00   |   0.06   |
| Class 1      |    0.42   |   1.00   |   0.40   |   0.57   |
| Class 2      |    0.42   |   0.08   |   0.83   |   0.15   |
+--------------+-----------+----------+----------+----------+
| Macro Average|    0.42   |   0.37   |   0.74   |   0.26   |
+--------------+-----------+----------+----

# Training + validation: 130

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

# # Function to read data from TSV file using pandas
# def read_tsv(file_path):
#     data = pd.read_table(file_path, names=['body', 'inflation'], header=0, dtype='object', engine='python')
#     return data

def read_csv_file(file_path):
    try:
        data = pd.read_csv(file_path, names=['body', 'inflation'], header=0, dtype='object')
        return data
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Test data file path (Replace with your Google Drive directory and file)
file_path = '/content/drive/MyDrive/world-inflation/data/reddit/production/test-data-200.csv'

# Read data from TSV file using pandas
test_data = read_csv_file(file_path)

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

# Encode the test data
test_encodings = tokenizer(test_data['body'].tolist(), truncation=True, padding=True)

# Convert the string labels to integers
test_labels = [int(label) for label in test_data['inflation']]

# Create the test dataset
test_dataset = TestDataset(test_encodings, test_labels)

# Initialize the model
model = RobertaForSequenceClassification.from_pretrained('/content/drive/MyDrive/world-inflation/data/model/roberta-large-fine-tuning-130/checkpoint-75/')

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=16)

# Lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Use the model to predict the labels of the test data
for batch in test_loader:
    inputs = {key: val.to(model.device) for key, val in batch.items() if key != 'labels'}
    labels = batch['labels'].to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    true_labels.extend(labels.tolist())
    predicted_labels.extend(predictions.tolist())

# Calculate and display accuracy, recall, precision, and F1 score
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average=None)
precision = precision_score(true_labels, predicted_labels, average=None)
f1 = f1_score(true_labels, predicted_labels, average=None)

# Display classification report and confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

# Display metrics for each class and macro/micro averages
macro_avg = precision.mean(), recall.mean(), f1.mean()
micro_avg = precision.sum() / 3, recall.sum() / 3, f1.sum() / 3

# Display metrics for each class and macro/micro averages
print("+--------------+-----------+----------+----------+----------+")
print("|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |")
print("+--------------+-----------+----------+----------+----------+")
for i in range(3):
    print(f"| Class {i}      |    {accuracy:.2f}   |   {recall[i]:.2f}   |   {precision[i]:.2f}   |   {f1[i]:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Macro Average|    {accuracy:.2f}   |   {recall.mean():.2f}   |   {precision.mean():.2f}   |   {f1.mean():.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Micro Average|    {accuracy:.2f}   |   {recall.sum()/3:.2f}   |   {precision.sum()/3:.2f}   |   {f1.sum()/3:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")


Classification Report:
              precision    recall  f1-score   support

           0       0.29      0.03      0.06        62
           1       0.38      0.92      0.54        77
           2       0.33      0.03      0.06        61

    accuracy                           0.38       200
   macro avg       0.33      0.33      0.22       200
weighted avg       0.34      0.38      0.24       200


Confusion Matrix:
[[ 2 59  1]
 [ 3 71  3]
 [ 2 57  2]]
+--------------+-----------+----------+----------+----------+
|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |
+--------------+-----------+----------+----------+----------+
| Class 0      |    0.38   |   0.03   |   0.29   |   0.06   |
| Class 1      |    0.38   |   0.92   |   0.38   |   0.54   |
| Class 2      |    0.38   |   0.03   |   0.33   |   0.06   |
+--------------+-----------+----------+----------+----------+
| Macro Average|    0.38   |   0.33   |   0.33   |   0.22   |
+--------------+-----------+----------+----

# Training + validation: 65

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

# # Function to read data from TSV file using pandas
# def read_tsv(file_path):
#     data = pd.read_table(file_path, names=['body', 'inflation'], header=0, dtype='object', engine='python')
#     return data

def read_csv_file(file_path):
    try:
        data = pd.read_csv(file_path, names=['body', 'inflation'], header=0, dtype='object')
        return data
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Test data file path (Replace with your Google Drive directory and file)
file_path = '/content/drive/MyDrive/world-inflation/data/reddit/production/test-data-200.csv'

# Read data from TSV file using pandas
test_data = read_csv_file(file_path)

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

# Encode the test data
test_encodings = tokenizer(test_data['body'].tolist(), truncation=True, padding=True)

# Convert the string labels to integers
test_labels = [int(label) for label in test_data['inflation']]

# Create the test dataset
test_dataset = TestDataset(test_encodings, test_labels)

# Initialize the model
model = RobertaForSequenceClassification.from_pretrained('/content/drive/MyDrive/world-inflation/data/model/roberta-large-fine-tuning-65/checkpoint-60/')

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=16)

# Lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Use the model to predict the labels of the test data
for batch in test_loader:
    inputs = {key: val.to(model.device) for key, val in batch.items() if key != 'labels'}
    labels = batch['labels'].to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    true_labels.extend(labels.tolist())
    predicted_labels.extend(predictions.tolist())

# Calculate and display accuracy, recall, precision, and F1 score
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average=None)
precision = precision_score(true_labels, predicted_labels, average=None)
f1 = f1_score(true_labels, predicted_labels, average=None)

# Display classification report and confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

# Display metrics for each class and macro/micro averages
macro_avg = precision.mean(), recall.mean(), f1.mean()
micro_avg = precision.sum() / 3, recall.sum() / 3, f1.sum() / 3

# Display metrics for each class and macro/micro averages
print("+--------------+-----------+----------+----------+----------+")
print("|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |")
print("+--------------+-----------+----------+----------+----------+")
for i in range(3):
    print(f"| Class {i}      |    {accuracy:.2f}   |   {recall[i]:.2f}   |   {precision[i]:.2f}   |   {f1[i]:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Macro Average|    {accuracy:.2f}   |   {recall.mean():.2f}   |   {precision.mean():.2f}   |   {f1.mean():.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Micro Average|    {accuracy:.2f}   |   {recall.sum()/3:.2f}   |   {precision.sum()/3:.2f}   |   {f1.sum()/3:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Classification Report:
              precision    recall  f1-score   support

           0       0.36      0.21      0.27        62
           1       0.40      0.56      0.46        77
           2       0.30      0.28      0.29        61

    accuracy                           0.36       200
   macro avg       0.35      0.35      0.34       200
weighted avg       0.36      0.36      0.35       200


Confusion Matrix:
[[13 31 18]
 [13 43 21]
 [10 34 17]]
+--------------+-----------+----------+----------+----------+
|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |
+--------------+-----------+----------+----------+----------+
| Class 0      |    0.36   |   0.21   |   0.36   |   0.27   |
| Class 1      |    0.36   |   0.56   |   0.40   |   0.46   |
| Class 2      |    0.36   |   0.28   |   0.30   |   0.29   |
+--------------+-----------+----------+----------+----------+
| Macro Average|    0.36   |   0.35   |   0.35   |   0.34   |
+--------------+-----------+----------+----

# DeBERTa: Training + validation: 1040

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

# # Function to read data from TSV file using pandas
# def read_tsv(file_path):
#     data = pd.read_table(file_path, names=['body', 'inflation'], header=0, dtype='object', engine='python')
#     return data

def read_csv_file(file_path):
    try:
        data = pd.read_csv(file_path, names=['body', 'inflation'], header=0, dtype='object')
        return data
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Test data file path (Replace with your Google Drive directory and file)
file_path = '/content/drive/MyDrive/world-inflation/data/reddit/production/test-data-200.csv'

# Read data from CSV file using pandas
test_data = read_csv_file(file_path)

# Initialize the tokenizer
tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-large')

# Encode the test data
test_encodings = tokenizer(test_data['body'].tolist(), truncation=True, padding=True)

# Convert the string labels to integers
test_labels = [int(label) for label in test_data['inflation']]

# Create the test dataset
test_dataset = TestDataset(test_encodings, test_labels)

# Initialize the model (Update path to your DeBERTaV3 checkpoint)
model = DebertaV2ForSequenceClassification.from_pretrained('/content/drive/MyDrive/world-inflation/data/model/deberta-large-fine-tuning-1040/checkpoint-588/')

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=16)

# Lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Use the model to predict the labels of the test data
for batch in test_loader:
    inputs = {key: val.to(model.device) for key, val in batch.items() if key != 'labels'}
    labels = batch['labels'].to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    true_labels.extend(labels.tolist())
    predicted_labels.extend(predictions.tolist())

# Calculate and display accuracy, recall, precision, and F1 score
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average=None)
precision = precision_score(true_labels, predicted_labels, average=None)
f1 = f1_score(true_labels, predicted_labels, average=None)

# Display classification report and confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

# Display metrics for each class and macro/micro averages
macro_avg = precision.mean(), recall.mean(), f1.mean()
micro_avg = precision.sum() / 3, recall.sum() / 3, f1.sum() / 3

# Display metrics for each class and macro/micro averages
print("+--------------+-----------+----------+----------+----------+")
print("|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |")
print("+--------------+-----------+----------+----------+----------+")
for i in range(3):
    print(f"| Class {i}      |    {accuracy:.2f}   |   {recall[i]:.2f}   |   {precision[i]:.2f}   |   {f1[i]:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Macro Average|    {accuracy:.2f}   |   {recall.mean():.2f}   |   {precision.mean():.2f}   |   {f1.mean():.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Micro Average|    {accuracy:.2f}   |   {recall.sum()/3:.2f}   |   {precision.sum()/3:.2f}   |   {f1.sum()/3:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.89      0.86        62
           1       0.77      0.69      0.73        77
           2       0.71      0.75      0.73        61

    accuracy                           0.77       200
   macro avg       0.77      0.78      0.77       200
weighted avg       0.77      0.77      0.77       200


Confusion Matrix:
[[55  6  1]
 [ 6 53 18]
 [ 5 10 46]]
+--------------+-----------+----------+----------+----------+
|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |
+--------------+-----------+----------+----------+----------+
| Class 0      |    0.77   |   0.89   |   0.83   |   0.86   |
| Class 1      |    0.77   |   0.69   |   0.77   |   0.73   |
| Class 2      |    0.77   |   0.75   |   0.71   |   0.73   |
+--------------+-----------+----------+----------+----------+
| Macro Average|    0.77   |   0.78   |   0.77   |   0.77   |
+--------------+-----------+----------+----

# DeBERTa: Training + validation: 65

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

# # Function to read data from TSV file using pandas
# def read_tsv(file_path):
#     data = pd.read_table(file_path, names=['body', 'inflation'], header=0, dtype='object', engine='python')
#     return data

def read_csv_file(file_path):
    try:
        data = pd.read_csv(file_path, names=['body', 'inflation'], header=0, dtype='object')
        return data
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Test data file path (Replace with your Google Drive directory and file)
file_path = '/content/drive/MyDrive/world-inflation/data/reddit/production/test-data-200.csv'

# Read data from CSV file using pandas
test_data = read_csv_file(file_path)

# Initialize the tokenizer
tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-large')

# Encode the test data
test_encodings = tokenizer(test_data['body'].tolist(), truncation=True, padding=True)

# Convert the string labels to integers
test_labels = [int(label) for label in test_data['inflation']]

# Create the test dataset
test_dataset = TestDataset(test_encodings, test_labels)

# Initialize the model (Update path to your DeBERTaV3 checkpoint)
model = DebertaV2ForSequenceClassification.from_pretrained('/content/drive/MyDrive/world-inflation/data/model/deberta-large-fine-tuning-65/checkpoint-96/')

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=16)

# Lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Use the model to predict the labels of the test data
for batch in test_loader:
    inputs = {key: val.to(model.device) for key, val in batch.items() if key != 'labels'}
    labels = batch['labels'].to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    true_labels.extend(labels.tolist())
    predicted_labels.extend(predictions.tolist())

# Calculate and display accuracy, recall, precision, and F1 score
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average=None)
precision = precision_score(true_labels, predicted_labels, average=None)
f1 = f1_score(true_labels, predicted_labels, average=None)

# Display classification report and confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

# Display metrics for each class and macro/micro averages
macro_avg = precision.mean(), recall.mean(), f1.mean()
micro_avg = precision.sum() / 3, recall.sum() / 3, f1.sum() / 3

# Display metrics for each class and macro/micro averages
print("+--------------+-----------+----------+----------+----------+")
print("|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |")
print("+--------------+-----------+----------+----------+----------+")
for i in range(3):
    print(f"| Class {i}      |    {accuracy:.2f}   |   {recall[i]:.2f}   |   {precision[i]:.2f}   |   {f1[i]:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Macro Average|    {accuracy:.2f}   |   {recall.mean():.2f}   |   {precision.mean():.2f}   |   {f1.mean():.2f}   |")
print("+--------------+-----------+----------+----------+----------+")
print(f"| Micro Average|    {accuracy:.2f}   |   {recall.sum()/3:.2f}   |   {precision.sum()/3:.2f}   |   {f1.sum()/3:.2f}   |")
print("+--------------+-----------+----------+----------+----------+")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Classification Report:
              precision    recall  f1-score   support

           0       0.43      0.61      0.51        62
           1       0.48      0.40      0.44        77
           2       0.40      0.31      0.35        61

    accuracy                           0.44       200
   macro avg       0.44      0.44      0.43       200
weighted avg       0.44      0.44      0.43       200


Confusion Matrix:
[[38 14 10]
 [28 31 18]
 [22 20 19]]
+--------------+-----------+----------+----------+----------+
|   Metric     | Accuracy  |  Recall  | Precision|  F1 Score |
+--------------+-----------+----------+----------+----------+
| Class 0      |    0.44   |   0.61   |   0.43   |   0.51   |
| Class 1      |    0.44   |   0.40   |   0.48   |   0.44   |
| Class 2      |    0.44   |   0.31   |   0.40   |   0.35   |
+--------------+-----------+----------+----------+----------+
| Macro Average|    0.44   |   0.44   |   0.44   |   0.43   |
+--------------+-----------+----------+----